In [1]:
import pandas as pd
import xarray as xr

## Analyse transport and power sectors in isolation for Spain and Germany

In [2]:
def read_variable(file_name, sheet_name, rows, sector_name):
    return (
        pd
        .read_excel(
            file_name, 
            sheet_name=sheet_name,
            skiprows=lambda x: x not in rows,
            usecols="A:W",
            index_col=0,
            header=0
        )
        .unstack()
        .rename("value")
        .rename_axis(index=["year", "country"])
        .to_frame()
        .assign(sector=sector_name)
        .reset_index()
        .set_index(["year", "country", "sector"])
        .to_xarray()["value"]
    )


def read_variable_from_all_sheets(file_name, sheet_to_sector_names, rows):
    return xr.concat(
        [
            read_variable(file_name, sheet_name, rows, sector_name) 
            for sheet_name, sector_name in sheet_to_sector_names.items()
        ],
        dim="sector"
    )

In [3]:
file_name = "../data/Germany Spain Data Power and Transpor Final 2021.04.28.xlsx"
sheet_to_sector_names = {
    "Power": "power",
    "Transport": "transport"
}
variables = xr.Dataset({
    "co2": read_variable_from_all_sheets(file_name, sheet_to_sector_names, rows=[2, 3, 4]),
    "energy": read_variable_from_all_sheets(file_name, sheet_to_sector_names, rows=[2, 7, 8]),
    "gdp": read_variable_from_all_sheets(file_name, sheet_to_sector_names, rows=[2, 11, 12]),
    "population": read_variable_from_all_sheets(file_name, sheet_to_sector_names, rows=[2, 15, 16])
})
variables

<xarray.Dataset>
Dimensions:     (country: 2, sector: 2, year: 22)
Coordinates:
  * year        (year) int64 1997 1998 1999 2000 2001 ... 2015 2016 2017 2018
  * country     (country) object 'Germany' 'Spain'
  * sector      (sector) object 'power' 'transport'
Data variables:
    co2         (year, country, sector) float64 3.508e+05 ... 8.921e+04
    energy      (year, country, sector) float64 5.654e+04 ... 3.253e+04
    gdp         (year, country, sector) float64 4.193e+04 8.895e+04 ... 5.01e+04
    population  (year, country, sector) int64 82012162 82012162 ... 46658447

In [4]:
def calculate_contributions(variables):
    kaya_factors = xr.Dataset({
        "carbon_intensity": variables["co2"] / variables["energy"],
        "energy_intensity": variables["energy"] / variables["gdp"],
        "gdp_per_capita": variables["gdp"] / variables["population"],
        "population": variables["population"]
    })
    return kaya_factors / kaya_factors.shift(year=1)

In [5]:
contributions = calculate_contributions(variables)
national_contributions = calculate_contributions(variables.sum("sector"))

In [6]:
contributions.to_dataframe().to_csv("../build/Germany-Spain-power-transport.csv", index=True, header=True)
national_contributions.to_dataframe().to_csv("../build/Germany-Spain-power-transport-national.csv", index=True, header=True)
variables.to_dataframe().to_csv("../build/Germany-Spain-power-transport-input.csv", index=True, header=True)

## Analyse primary, secondary, tertiary sectors in combination for Spain and Germany

In [7]:
file_name = "../data/Germany Spain Data Final 2021.04.28.xlsx"
sheet_to_sector_names = {
    "primary": "primary",
    "secondary": "secondary",
    "tertiary": "tertiary"
}
variables = xr.Dataset({
    "co2": read_variable_from_all_sheets(file_name, sheet_to_sector_names, rows=[2, 3, 4]),
    "energy": read_variable_from_all_sheets(file_name, sheet_to_sector_names, rows=[2, 7, 8]),
    "gdp": read_variable_from_all_sheets(file_name, sheet_to_sector_names, rows=[2, 11, 12]),
    #"population": read_variable(file_name, sheet_name="total", rows=[3, 16, 17], sector_name="total").squeeze("sector")
})
variables

<xarray.Dataset>
Dimensions:  (country: 2, sector: 3, year: 22)
Coordinates:
  * year     (year) int64 1997 1998 1999 2000 2001 ... 2014 2015 2016 2017 2018
  * country  (country) object 'Germany' 'Spain'
  * sector   (sector) object 'primary' 'secondary' 'tertiary'
Data variables:
    co2      (year, country, sector) float64 7.533e+03 1.393e+05 ... 1.247e+04
    energy   (year, country, sector) float64 2.047e+03 5.231e+04 ... 1.119e+04
    gdp      (year, country, sector) float64 1.926e+04 6.679e+05 ... 7.945e+05

In [8]:
def calculate_contributions(variables):
    kaya_factors = xr.Dataset({
        "gdp_share": variables["gdp"] / variables["gdp"].sum("sector"),
        "carbon_intensity": variables["co2"] / variables["energy"],
        "energy_intensity": variables["energy"] / (variables["gdp"] / variables["gdp"].sum("sector")),
        "gdp": variables["gdp"]
    })
    return kaya_factors / kaya_factors.shift(year=1)


def calculate_national_contributions(variables):
    kaya_factors = xr.Dataset({
        "carbon_intensity": variables["co2"] / variables["energy"],
        "energy_intensity": variables["energy"] / variables["gdp"],
        "gdp": variables["gdp"]
    })
    return kaya_factors / kaya_factors.shift(year=1)

In [9]:
contributions = calculate_contributions(variables)
national_contributions = calculate_national_contributions(variables.sum("sector"))

In [10]:
contributions.to_dataframe().to_csv("../build/Germany-Spain-primary-secondary-tertiary.csv", index=True, header=True)
national_contributions.to_dataframe().to_csv("../build/Germany-Spain-primary-secondary-tertiary-national.csv", index=True, header=True)
variables.to_dataframe().to_csv("../build/Germany-Spain-primary-secondary-tertiary-data.csv", index=True, header=True)